In [1]:
import math
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point

In [2]:
# Function to calculate the distance between two waypoints
def distance(lat1, lon1, alt1, lat2, lon2, alt2):
    R = 6371  # Earth radius in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + \
        math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * \
        math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    dz = abs(alt2 - alt1) * 0.3048  # Convert altitude difference from ft to m
    return math.sqrt(d ** 2 + dz ** 2)

# Function to check for potential conflicts between two UAV flight plans
def check_conflict(flt_pln_1, flt_pln_2):
    for i in range(len(flt_pln_1)):
        for j in range(len(flt_pln_2)):
            dist = distance(flt_pln_1[i][0], flt_pln_1[i][1], flt_pln_1[i][2],
                             flt_pln_2[j][0], flt_pln_2[j][1], flt_pln_2[j][2])
            if dist < 1:
                return True
    return False

# Sample flight plans for two UAVs
uav1_flt_pln = [(40.7128, -74.0060, 50), (41.8781, -87.6298, 100), (37.7749, -122.4194, 75)]
uav2_flt_pln = [(34.0522, -118.2437, 150), (39.7392, -104.9903, 125), (37.3382, -121.8863, 50)]

# Check for potential conflicts
if check_conflict(uav1_flt_pln, uav2_flt_pln):
    print("Potential conflict detected!")
else:
    print("No potential conflicts detected.")

No potential conflicts detected.


In [4]:
import numpy as np
from shapely.geometry import LineString, Point

def get_dummy_rt(waypoints):
    # Create pairs of waypoints that cross each other
    dummy_rt = []
    for i in range(len(waypoints)):
        for j in range(i+1, len(waypoints)):
            if i != j-1 and i != j+1:
                continue
            rt1 = (waypoints[i], waypoints[j])
            for k in range(len(waypoints)):
                if k == i or k == j:
                    continue
                rt2 = (waypoints[k], waypoints[k])
                if LineString(rt1).intersects(LineString(rt2)):
                    dummy_rt.append(rt1 + rt2)
    return dummy_rt

def get_intersec_points(dummy_rt):
    # Calculate intersec points for all pairs of dummy routes
    intersec_points = []
    for i in range(len(dummy_rt)):
        for j in range(i+1, len(dummy_rt)):
            ln1 = LineString(dummy_rt[i])
            ln2 = LineString(dummy_rt[j])
            intersec = ln1.intersec(ln2)
            if isinstance(intersec, Point):
                intersec_points.append((intersec.x, intersec.y))
    return intersec_points

def get_cells(min_lat, max_lat, min_lon, max_lon, cell_width):
    # Divide the airspace into a grid of cells
    num_cols = int(np.ceil((max_lon - min_lon) / cell_width))
    num_rows = int(np.ceil((max_lat - min_lat) / cell_width))
    cells = []
    for i in range(num_rows):
        for j in range(num_cols):
            cell_min_lon = min_lon + j * cell_width
            cell_min_lat = min_lat + i * cell_width
            cell_max_lon = min_lon + (j+1) * cell_width
            cell_max_lat = min_lat + (i+1) * cell_width
            cells.append((cell_min_lon, cell_min_lat, cell_max_lon, cell_max_lat))
    return cells

def get_potential_congestion_areas(cells, intersec_points, radius, percentile):
    # Count the number of intersec points in each cell
    cell_counts = np.zeros(len(cells))
    for point in intersec_points:
        for i, cell in enumerate(cells):
            if cell[0] <= point[0] < cell[2] and cell[1] <= point[1] < cell[3]:
                cell_counts[i] += 1
                break
    
    # Calculate the threshold for potential congestion areas
    threshold = np.percentile(cell_counts, percentile)
    
def identify_congestion_areas(routes):
    # Get all unique waypoints from all routes
    waypoints = set()
    for route in routes:
        waypoints.update(route)
    waypoints = list(waypoints)

    # Create grid cells with a radius of 5km
    cells = get_cells(min_lat=min([wp[0] for wp in waypoints]), 
                      max_lat=max([wp[0] for wp in waypoints]), 
                      min_lon=min([wp[1] for wp in waypoints]), 
                      max_lon=max([wp[1] for wp in waypoints]), 
                      radius=5000, resolution=0.05)

    # Calculate the total number of crossings for each cell
    cell_crossings = {}
    for cell in cells:
        crossings = 0
        for route in routes:
            for i in range(len(route) - 1):
                if intersects(cell, route[i], route[i+1]):
                    crossings += 1
        cell_crossings[cell] = crossings

    # Identify congestion areas with more than 25% of total crossings
    congestion_areas = []
    total_crossings = sum(cell_crossings.values())
    for cell, crossings in cell_crossings.items():
        if crossings / total_crossings >= 0.25:
            congestion_areas.append((cell, crossings))

    return congestion_areas


In [ ]:
class Waypoint:
    def __init__(self, lat, lon, alt):
        self.lat = lat
        self.lon = lon
        self.alt = alt

class FlightPlan:
    def __init__(self, waypoints):
        self.waypoints = waypoints

def distance(wp1, wp2):
    # calculate distance between two waypoints using Haversine formula
    ...

def crossings(plan, cell):
    count = 0
    for i in range(len(plan.waypoints) - 1):
        wp1 = plan.waypoints[i]
        wp2 = plan.waypoints[i+1]
        if wp1.alt <= cell.altitude and wp2.alt >= cell.altitude:
            # calculate intersection of flight segment and cell altitude
            dalt = cell.altitude - wp1.alt
            lat1 = math.radians(wp1.lat)
            lon1 = math.radians(wp1.lon)
            lat2 = math.radians(wp2.lat)
            lon2 = math.radians(wp2.lon)
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
            c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
            dist = 6371 * c * 1000
            if dist <= 5000:
                # calculate intersection point
                f = dist / math.sqrt(5000**2 - dist**2)
                lat_int = math.degrees(lat1 + f * (lat2 - lat1))
                lon_int = math.degrees(lon1 + f * (lon2 - lon1))
                # check if intersection point is within cell boundaries
                if (cell.lat - 0.5*cell.width) <= lat_int <= (cell.lat + 0.5*cell.width) and \
                   (cell.lon - 0.5*cell.width) <= lon_int <= (cell.lon + 0.5*cell.width):
                    count += 1
    return count

class Cell:
    def __init__(self, lat, lon, altitude, width):
        self.lat = lat
        self.lon = lon
        self.altitude = altitude
        self.width = width
        self.crossings = 0

def get_flight_plans():
    plans = []
    while True:
        print("Enter flight plan as comma-separated latitudes, longitudes, and altitudes (e.g., 40.0,-120.0,1000.0):")
        line = input()
        if line.strip() == "":
            break
        waypoints = []
        for wp_str in line.split(","):
            lat, lon, alt = [float(x) for x in wp_str.split(",")]
            waypoints.append(Waypoint(lat, lon, alt))
        plans.append(FlightPlan(waypoints))
    return plans
def get_cells(min_lat, max_lat, min_lon, max_lon, altitude, width):
    cells = []
    nlat = int((max_lat - min_lat) / width)
    nlon = int((max_lon - min_lon) / width)
    for i in range(nlat):
        for j in range(nlon):
            lat = min_lat + (i + 0.5) * width
            lon = min_lon + (j + 0.5) * width
            cells.append(Cell(lat, lon, altitude, width))
    return cells

def plot_cells(cells):
    lats = [cell.lat for cell in cells]
    lons = [cell.lon for cell in cells]
    plt.scatter(lons, lats, s=20, c=[cell.crossings for cell in cells], cmap='cool')
    plt.colorbar()
    plt.show()

def main():
    # get flight plans from user input
    plans = get_flight_plans()

    # define grid cells for airspace
    cells = get_cells(35.0, 42.0, -120.0, -110.0, 1000, 0.05)

    # count number of flight plan crossings for each cell
    for cell in cells:
        for plan in plans:
            cell.crossings += crossings(plan, cell)

    # apply DBSCAN clustering to identify congestion areas
    data = np.array([[cell.lon, cell.lat] for cell in cells])
    db = DBSCAN(eps=5, min_samples=10).fit(data)
    labels = db.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    print("Number of congestion areas:", n_clusters)

    # calculate number of crossings per cluster
    cluster_counts = [0] * n_clusters
    for i in range(len(cells)):
        if labels[i] != -1:
            cluster_counts[labels[i]] += cells[i].crossings

    # calculate 25th percentile of crossings
    crossings_threshold = np.percentile(cluster_counts, 25)
    print("25th percentile of crossings:", crossings_threshold)

    # mark congestion areas with crossings above threshold
    for i in range(len(cells)):
        if labels[i] != -1 and cluster_counts[labels[i]] > crossings_threshold:
            cells[i].congested = True

    # plot grid cells with color representing number of crossings
    plot_cells(cells)

if __name__ == '__main__':
    main()

Enter flight plan as comma-separated latitudes, longitudes, and altitudes (e.g., 40.0,-120.0,1000.0):
